<a href="https://colab.research.google.com/github/HenryZumaeta/ReprojectCoordinates/blob/main/ReprojectCoordinates_WGS84_to_PSAD56.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **<font color='DD1C30'>Reproject Coordinates WGS84 to PSAD56 - Shared by [Henry Zumaeta](https://github.com/HenryZumaeta)</font>**

<center>
  <img src='https://raw.githubusercontent.com/HenryZumaeta/DataProgrammingRepo/main/db/WGS84_to_PSAD56.png' height="200" alt="WGS84_to_PSAD56-logo"/>
</center>

---


<center>
  <img src='https://raw.githubusercontent.com/HenryZumaeta/DataProgrammingRepo/main/db/Coordenadas1.png' height="400" alt="Coordenadas-img"/>
</center>

**<font color='DD1C30'><h2>Descripción:</h2></font>**

<font color='1E90FF'><h4>El archivo CSV debe tener las columnas con nombre de X y Y.</h4></font>

<font color='1E90FF'><h4>Donde:</h4></font>

<font color='1E90FF'><h4>Y = Norte(metros) o Latitud(grados)</h4></font>

<font color='1E90FF'><h4>X = Este(metros) o Longitud(grados)</h4></font>

<font color='1E90FF'><h4>Este código carga un archivo CSV con coordenadas, luego convierte estas coordenadas entre sistemas de referencia (WGS84 a PSAD56 o viceversa tanto de UTM a Geográficas o viceversa). Las coordenadas convertidas se guardan en un nuevo archivo CSV que contiene solo las columnas originales y las columnas convertidas. El archivo resultante se descarga automáticamente al finalizar el proceso.</h4></font>

---



In [ ]:
# @markdown **<font color='DD1C30'><h3>Subir archivo CSV</h3></font>**

# Librerías
from google.colab import files
import pandas as pd
from pyproj import CRS, Transformer

# Carga del archivo
uploaded = files.upload()

for filename in uploaded.keys():
    df = pd.read_csv(filename)
    print(f"Archivo {filename} subido y cargado en un DataFrame")
    print(df.head())

Saving CoordenadasUTM_17S_PSAD56.csv to CoordenadasUTM_17S_PSAD56.csv
Archivo CoordenadasUTM_17S_PSAD56.csv subido y cargado en un DataFrame
             X            Y
0  474002.6373  9530109.350
1  474002.6373  9530124.399
2  474003.7264  9530124.399
3  474003.7264  9530159.894
4  474003.7264  9530171.231


In [ ]:
# @title  { display-mode: "form" }

# @markdown **<font color='DD1C30'><h3>SCR de entrada: Coordenadas Inicio</h3></center></font>**<br>
# Parámetros de entrada
ENTRADA_SISTEMA = "UTM"  # @param ["UTM", "GEOGRAFICAS"]
ENTRADA_ZONA = "17S"  # @param ["17S", "18S", "19S"]
ENTRADA_DATUM = "PSAD56"  # @param ["WGS84", "PSAD56"]

# @markdown **<font color='DD1C30'><h3>SCR Objetivo: Coordenadas transformadas</h3></center></font>**<br>
# Parámetros de salida
SALIDA_SISTEMA = "UTM"  # @param ["UTM", "GEOGRAFICAS"]
SALIDA_ZONA = "17S"  # @param ["17S", "18S", "19S"]
SALIDA_DATUM = "WGS84"  # @param ["WGS84", "PSAD56"]

# Definir nombres de columnas basados en el sistema de salida
if SALIDA_SISTEMA == "UTM":
    colX = "ESTE"
    colY = "NORTE"
else:
    colX = "LATITUD"
    colY = "LONGITUD"

# Definir el nombre del archivo de salida
if ENTRADA_SISTEMA == "GEOGRAFICAS" and SALIDA_SISTEMA == "GEOGRAFICAS":
    name_file = ENTRADA_SISTEMA + "_" + ENTRADA_DATUM + "_A_" + SALIDA_SISTEMA + "_" + SALIDA_DATUM + ".csv"
elif ENTRADA_SISTEMA == "GEOGRAFICAS" and SALIDA_SISTEMA == "UTM":
    name_file = ENTRADA_SISTEMA + "_" + ENTRADA_DATUM + "_A_" + SALIDA_SISTEMA + "_" + SALIDA_DATUM + "_" + SALIDA_ZONA + ".csv"
elif ENTRADA_SISTEMA == "UTM" and SALIDA_SISTEMA == "GEOGRAFICAS":
    name_file = ENTRADA_SISTEMA + "_" + ENTRADA_DATUM + "_" + ENTRADA_ZONA + "_A_" + SALIDA_SISTEMA + "_" + SALIDA_DATUM + ".csv"
else:
    name_file = ENTRADA_SISTEMA + "_" + ENTRADA_DATUM + "_" + ENTRADA_ZONA + "_A_" + SALIDA_SISTEMA + "_" + SALIDA_DATUM + "_" + SALIDA_ZONA + ".csv"

# Imprimir los valores de entrada y salida
print(f"Sistema de entrada: {ENTRADA_SISTEMA}")
print(f"Zona de entrada: {ENTRADA_ZONA}")
print(f"Datum de entrada: {ENTRADA_DATUM}")
print(f"Sistema de salida: {SALIDA_SISTEMA}")
print(f"Zona de salida: {SALIDA_ZONA}")
print(f"Datum de salida: {SALIDA_DATUM}")
print(f"Nombre del archivo de salida: {name_file}")

# Función para definir el CRS
def definir_crs(sistema, zona, datum):
    if sistema == "UTM":
        zone_number = zona[:-1]
        if datum == "WGS84":
            crs = f"+proj=utm +zone={zone_number} +south +datum=WGS84 +units=m +no_defs"
        elif datum == "PSAD56":
            crs = f"+proj=utm +zone={zone_number} +south +ellps=intl +towgs84=-288,175,-376,0,0,0,0 +units=m +no_defs"
    else:
        if datum == "WGS84":
            crs = "EPSG:4326"
        elif datum == "PSAD56":
            crs = "EPSG:4248"
    return crs

# Función para convertir coordenadas
def convertir_coordenadas(df, entrada_sistema, entrada_zona, entrada_datum, salida_sistema, salida_zona, salida_datum):
    crs_in = definir_crs(entrada_sistema, entrada_zona, entrada_datum)
    crs_out = definir_crs(salida_sistema, salida_zona, salida_datum)

    # Crear un transformador
    transformer = Transformer.from_crs(CRS(crs_in), CRS(crs_out))

    # Convertir coordenadas
    if entrada_sistema == "UTM" and salida_sistema == "GEOGRAFICAS":
        df[colX], df[colY] = transformer.transform(df['X'].values, df['Y'].values)
    elif entrada_sistema == "UTM" and salida_sistema == "UTM":
        df[colX], df[colY] = transformer.transform(df['X'].values, df['Y'].values)
    else:
        df[colX], df[colY] = transformer.transform(df['Y'].values, df['X'].values)

    return df

# Uso de la función para convertir coordenadas
df_convertido = convertir_coordenadas(df, ENTRADA_SISTEMA, ENTRADA_ZONA, ENTRADA_DATUM, SALIDA_SISTEMA, SALIDA_ZONA, SALIDA_DATUM)
print("Coordenadas convertidas")

# Columnas originales y las convertidas
df_salida = df[['X', 'Y', colX, colY]]

# Exportar
df_salida.to_csv(name_file, index=False)
print(f"Archivo guardado como {name_file}")

# Descarga
files.download(name_file)


Sistema de entrada: UTM
Zona de entrada: 17S
Datum de entrada: PSAD56
Sistema de salida: UTM
Zona de salida: 17S
Datum de salida: WGS84
Nombre del archivo de salida: UTM_PSAD56_17S_A_UTM_WGS84_17S.csv
Coordenadas convertidas
Archivo guardado como UTM_PSAD56_17S_A_UTM_WGS84_17S.csv


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# <font color='DD1C30'><img src="https://media2.giphy.com/media/v1.Y2lkPTc5MGI3NjExbHduN3M0MWw1MThzYWVwaGw3cDFsZG92ODJ5Yzc4b3lnNjM4Y2ptMyZlcD12MV9pbnRlcm5hbF9naWZfYnlfaWQmY3Q9Zw/qZb3YFfAf8YRgWqaVV/giphy.webp" width="7%"> <strong>Support My Work</strong></font>

<p align="left">
  <a href="https://www.buymeacoffee.com/henryzumaeta" target="_blank">
    <img src="https://raw.githubusercontent.com/HenryZumaeta/DataProgrammingRepo/main/db/BuyMeCoffee_77px.png" alt="Buy Me A Coffee" style="height: 42px; width: 151.9px;">
  </a>
  <a href="https://raw.githubusercontent.com/HenryZumaeta/DataProgrammingRepo/main/db/QR_yape.jpg" target="_blank">
    <img src="https://raw.githubusercontent.com/HenryZumaeta/DataProgrammingRepo/main/db/Yape_77px.png" alt="Yape" style="height: 42px; width: 151.9px;">
  </a>
</p>

# <font color='DD1C30'><img src="https://media4.giphy.com/media/v1.Y2lkPTc5MGI3NjExaXQ5eHVhZGx4djZhcW42ZDR3Y3dnaWY5NjNwbGs3MGt0YXl1cWM0cSZlcD12MV9pbnRlcm5hbF9naWZfYnlfaWQmY3Q9cw/1MyJr6oe9sDCCXs5ti/giphy.webp" width="8%"> <strong>Connect With Me</strong></font>

<p align="left">
  <a href="https://www.linkedin.com/in/henryzumaeta/" target="_blank"><img align="center" src="https://img.icons8.com/?size=100&id=64154&format=png&color=000000" alt="linkedin" height="50" width="50" /></a>
  <a href="https://twitter.com/henryzumaeta" target="_blank"><img align="center" src="https://img.icons8.com/?size=100&id=64156&format=png&color=000000" alt="twitter" height="50" width="50" /></a>
  <a href="https://www.instagram.com/henryzumaeta/" target="_blank"><img align="center" src="https://img.icons8.com/?size=100&id=hFoVFpm6gl9A&format=png&color=000000" alt="instagram" height="50" width="50" /></a>
  <a href="https://api.whatsapp.com/send?phone=51963719768&text=%F0%9F%91%8B%F0%9F%91%A8%E2%80%8D%F0%9F%92%BB" target="_blank"><img align="center" src="https://img.icons8.com/?size=100&id=108636&format=png&color=000000" alt="whatsapp" height="50" width="50" /></a>
  <a href="mailto:henry.zumaeta.l@uni.pe" target="_blank"><img align="center" src="https://img.icons8.com/?size=100&id=6QtoKjRma1Cq&format=png&color=000000" alt="email" height="50" width="50" /></a>
</p>

# <font color='DD1C30'><img src="https://media2.giphy.com/media/v1.Y2lkPTc5MGI3NjExeHM0YnNpMnJrYWJrd243c2J5bW41d2Z4c2xoNzM2YjU0N3dsZmhvayZlcD12MV9pbnRlcm5hbF9naWZfYnlfaWQmY3Q9Zw/Xd1RbiqD2eL0YCHIjv/giphy.webp" width="6%"> <strong>Advertencia</strong></font>

<font color='FFD700'><h4>El autor no se hace responsable del mal uso del formulario y script.</h4></font>
